In [ ]:
!pip install -qqq pandas pycaret linear-tree shap sdv explainerdashboard

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.1.4 which is incompatible.
xarray-einstats 0.4.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
xarray-einstats 0.4.0 requires scipy>=1.6, but you have scipy 1.5.4 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
tensorflow 2.9.2 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jax 0.3.25 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.9 which is incompatible.
cmdsta

In [ ]:
!pip install numpy==1.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.0-cp38-cp38-manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.0 which is incompatible.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.1.4 which is incompatible.
xarray-einstats 0.4.0 requires scipy>=1.6, but you have scipy 1.5.4 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.9 which is incompatible.
cmdstanpy 1.0.8 requires numpy>=1.

In [ ]:
import pycaret.regression as py_reg
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_error
from lineartree import LinearTreeRegressor
# import shap
import matplotlib.pyplot as plt

from sdv.tabular import GaussianCopula, TVAE, CTGAN, CopulaGAN

from pandas_profiling import ProfileReport

import numpy as np
import pandas as pd
import seaborn as sns
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Irini/NEW"

Mounted at /content/drive


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):

  err = (y_true - y_pred)/y_true
  err = err[~np.isinf(err)]  # drop out divison by zero errors 
  abs_err = abs(err)
  mape = 1/len(err) * sum(abs_err)

  return mape

In [ ]:
# load dataset into memory
df = pd.read_csv(f"{path}/NEW_v00_imputed_SMOTE.csv")

df.head()

,NMs,Coating,Crystallinity_%,Aver_crystallite_sizes_nm,Core_size_nm_TEM,Spherical_surface_Area_nm^2 _TEM,Na 1s_Concent_%,O 1s_Concent_%,Ag 3d_Concent_%,C 1s_Concent_%,N 1s_Concent_%,Hydro_size_t0_nm,PI_t0,Hydro_size_t24_nm,PI_t24,Exposure dose_PPM,Organ,Cell_line,multiwell,Assay,viability_%,viability
0,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,254.33,0.38,113.96667,0.5,100.0,Lung,A549,MW6,Alamar_Blue,2.99,very_toxic
1,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,254.33,0.38,113.96667,0.5,100.0,Lung,A549,MW6,Alamar_Blue,3.06,very_toxic
2,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,254.33,0.38,113.96667,0.5,100.0,Lung,A549,MW6,Alamar_Blue,3.09,very_toxic
3,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,173.53,0.42,250.53334,0.4,50.0,Lung,A549,MW6,Alamar_Blue,3.11,very_toxic
4,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,173.53,0.42,250.53334,0.4,50.0,Lung,A549,MW6,Alamar_Blue,3.13,very_toxic


In [ ]:
df["N 1s_Concent_%"] = df["N 1s_Concent_%"].astype(float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NMs                               1682 non-null   object 
 1   Coating                           1682 non-null   object 
 2   Crystallinity_%                   1682 non-null   float64
 3   Aver_crystallite_sizes_nm         1682 non-null   float64
 4   Core_size_nm_TEM                  1682 non-null   float64
 5   Spherical_surface_Area_nm^2 _TEM  1682 non-null   float64
 6   Na 1s_Concent_%                   1682 non-null   float64
 7   O 1s_Concent_%                    1682 non-null   float64
 8   Ag 3d_Concent_%                   1682 non-null   float64
 9   C 1s_Concent_%                    1682 non-null   float64
 10  N 1s_Concent_%                    1682 non-null   float64
 11  Hydro_size_t0_nm                  1682 non-null   float64
 12  PI_t0 

In [ ]:
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_notebook_iframe()

In [ ]:
df.columns

Index(['NMs', 'Coating', 'Crystallinity_%', 'Aver_crystallite_sizes_nm',
       'Core_size_nm_TEM', 'Spherical_surface_Area_nm^2 _TEM',
       'Na 1s_Concent_%', 'O 1s_Concent_%', 'Ag 3d_Concent_%',
       'C 1s_Concent_%', 'N 1s_Concent_%', 'Hydro_size_t0_nm', 'PI_t0',
       'Hydro_size_t24_nm', 'PI_t24', 'Exposure dose_PPM', 'Organ',
       'Cell_line', 'multiwell', 'Assay', 'viability_%', 'viability'],
      dtype='object')

In [ ]:
# set the target variable
target = "viability_%"

ignore_features = ["NMs",
                   "Organ",
                   "multiwell",
                   "viability"]

# set up dependent variables
categorical_features = ["Coating",
                        "Organ",
                        "Cell_line",
                        "multiwell", 
                        "Assay"]

impute = []

numeric_features = ['Crystallinity_%', 
                    'Aver_crystallite_sizes_nm',
                    'Spherical_surface_Area_nm^2 _TEM',
                    'O 1s_Concent_%', 
                    'Ag 3d_Concent_%',
                    'C 1s_Concent_%', 
                    'N 1s_Concent_%', 
                    'Hydro_size_t0_nm',
                    'PI_t0',
                    'Hydro_size_t24_nm',
                    'PI_t24', 
                    'Exposure dose_PPM']

In [ ]:
preprocess = True

normalize = False
normalize_method = "robust"
remove_outliers = False
remove_perfect_collinearity = False
remove_multicollinearity = False
ignore_low_variance = False

impute = False
impute_method = "iterative"

polynomial_features = False
polynomial_degree = 2

trigonometry_features = False

feature_interaction = True
feature_ratio = True

combine_rare_levels = False

feature_selection = False
feature_selection_method = "boruta"

experiment_setup = py_reg.setup(data=df,
                                target=target,
                                categorical_features=categorical_features,
                                numeric_features=numeric_features,
                                ignore_features=ignore_features,
                                ignore_low_variance=ignore_low_variance,
                                remove_outliers=remove_outliers,
                                remove_multicollinearity=remove_multicollinearity,
                                multicollinearity_threshold=0.9,
                                remove_perfect_collinearity=remove_perfect_collinearity,
                                preprocess=preprocess,
                                normalize=normalize,
                                normalize_method=normalize_method,
                                imputation_type=impute_method,
                                combine_rare_levels=combine_rare_levels,
                                polynomial_features=polynomial_features,
                                polynomial_degree=polynomial_degree,
                                trigonometry_features=trigonometry_features,
                                feature_interaction=feature_interaction,
                                feature_ratio=feature_ratio,
                                feature_selection=feature_selection,
                                feature_selection_method=feature_selection_method,
                                use_gpu=True,
                                silent=True)

,Description,Value
0,session_id,5453
1,Target,viability_%
2,Original Data,"(1682, 22)"
3,Missing Values,False
4,Numeric Features,14
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1177, 40)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['Coating',
                                                            'Cell_line',
                                                            'Assay'],
                                      display_types=False,
                                      features_todrop=['NMs', 'Organ',
                                                       'multiwell',
                                                       'viability'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['Crystallinity_%',
                                                          'Aver_crystallite_sizes_nm',
                                                          'Spherical_surface_Area_nm^2 '
                          

In [ ]:
py_reg.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.252570e+01,3.801413e+02,1.933450e+01,7.195000e-01,0.7858,2.480700e+00,0.145
gbr,Gradient Boosting Regressor,1.329800e+01,3.817764e+02,1.941800e+01,7.189000e-01,0.8195,2.624700e+00,0.339
rf,Random Forest Regressor,1.258210e+01,3.958791e+02,1.971140e+01,7.074000e-01,0.7980,2.524400e+00,1.158
et,Extra Trees Regressor,1.251340e+01,4.257782e+02,2.040400e+01,6.857000e-01,0.8027,2.488900e+00,0.626
dt,Decision Tree Regressor,1.304390e+01,4.757268e+02,2.157080e+01,6.486000e-01,0.8355,2.400100e+00,0.024
ada,AdaBoost Regressor,1.731350e+01,5.131215e+02,2.261320e+01,6.237000e-01,0.9225,3.021000e+00,0.064
ridge,Ridge Regression,1.679220e+01,5.253128e+02,2.286000e+01,6.141000e-01,0.9121,2.724100e+00,0.025
br,Bayesian Ridge,1.695940e+01,5.264827e+02,2.288940e+01,6.133000e-01,0.9216,2.805000e+00,0.027
lasso,Lasso Regression,1.775000e+01,5.407873e+02,2.321530e+01,6.033000e-01,0.9548,3.121800e+00,0.025
lr,Linear Regression,1.680660e+01,5.758854e+02,2.391240e+01,5.766000e-01,0.8967,2.548700e+00,0.011


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=5453, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:compare_models() succesfully completed......................................


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=5453, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
model = py_reg.create_model("lightgbm")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13.0210,362.8833,19.0495,0.6939,0.6374,1.0460
1,11.9836,327.1305,18.0867,0.7648,0.8972,3.2214
2,12.6223,386.5662,19.6613,0.6944,0.8521,3.2695
3,10.8628,231.2104,15.2056,0.8480,0.7137,1.5745
4,10.3974,268.6802,16.3915,0.8230,0.6744,1.3669
5,12.9043,472.9354,21.7471,0.6776,0.9504,3.6544
6,15.2028,610.9565,24.7175,0.5585,1.0391,4.9407
7,13.0543,378.5734,19.4570,0.7176,0.6915,1.4132
8,13.6376,406.4968,20.1618,0.6919,0.7040,2.4023


INFO:logs:create_model_container: 21
INFO:logs:master_model_container: 21
INFO:logs:display_container: 5
INFO:logs:LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=5453, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:create_model() succesfully completed......................................


In [ ]:
model = py_reg.tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13.3310,361.6793,19.0179,0.6949,0.6262,1.1164
1,11.8613,307.3331,17.5309,0.7790,0.8991,3.0610
2,12.7953,397.2119,19.9302,0.6860,0.8678,3.1065
3,10.9606,243.4240,15.6021,0.8399,0.7172,1.5383
4,11.0776,283.5458,16.8388,0.8132,0.6833,1.3283
5,12.9984,429.9490,20.7352,0.7069,0.8982,3.3964
6,14.7015,523.7986,22.8866,0.6215,1.0192,4.5988
7,12.5535,337.8803,18.3815,0.7480,0.7090,1.3543
8,14.1569,434.6923,20.8493,0.6705,0.7560,2.5743


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 6
INFO:logs:LGBMRegressor(bagging_fraction=0.9, bagging_freq=7, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.4, max_depth=-1,
              min_child_samples=86, min_child_weight=0.001, min_split_gain=0.3,
              n_estimators=100, n_jobs=-1, num_leaves=200, objective=None,
              random_state=5453, reg_alpha=0.2, reg_lambda=0.1, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:tune_model() succesfully completed......................................


In [ ]:
py_reg.evaluate_model(model)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=LGBMRegressor(bagging_fraction=0.9, bagging_freq=7, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.4, max_depth=-1,
              min_child_samples=86, min_child_weight=0.001, min_split_gain=0.3,
              n_estimators=100, n_jobs=-1, num_leaves=200, objective=None,
              random_state=5453, reg_alpha=0.2, reg_lambda=0.1, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
# py_reg.dashboard(model, display_format="inline")

In [ ]:
model = py_reg.finalize_model(model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=LGBMRegressor(bagging_fraction=0.9, bagging_freq=7, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.4, max_depth=-1,
              min_child_samples=86, min_child_weight=0.001, min_split_gain=0.3,
              n_estimators=100, n_jobs=-1, num_leaves=200, objective=None,
              random_state=5453, reg_alpha=0.2, reg_lambda=0.1, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0), fit_kwargs=None, groups=None, model_only=True, display=None, experiment_custom_tags=None, return_train_score=False)
INFO:logs:Finalizing LGBMRegressor(bagging_fraction=0.9, bagging_freq=7, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.4, max_depth=-1,
              min_child_samples=86

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 

INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Finalizing model


[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 5
INFO:logs:LGBMRegressor(bagging_fraction=0.9, bagging_freq=7, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.4, max_depth=-1,
              min_child_samples=86, min_child_weight=0.001, min_split_gain=0.3,
              n_estimators=100, n_jobs=-1, num_leaves=200, objective=None,
              random_state=5453, reg_alpha=0.2, reg_lambda=0.1, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:create_model() succesfully completed......................................
INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 4
INFO:logs:LGBMRegressor(bagging_fraction=0.9, bagging_freq=7, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
    

In [ ]:
py_reg.save_model(model, "nano_viability_regressor")

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=LGBMRegressor(bagging_fraction=0.9, bagging_freq=7, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
              importance_type='split', learning_rate=0.4, max_depth=-1,
              min_child_samples=86, min_child_weight=0.001, min_split_gain=0.3,
              n_estimators=100, n_jobs=-1, num_leaves=200, objective=None,
              random_state=5453, reg_alpha=0.2, reg_lambda=0.1, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0), model_name=nano_viability_regressor, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['Coating',
                                                            'Cell_line',
                                                            'Assay'],
                                      display_types=False,
                                    

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['Coating',
                                                             'Cell_line',
                                                             'Assay'],
                                       display_types=False,
                                       features_todrop=['NMs', 'Organ',
                                                        'multiwell',
                                                        'viability'],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['Crystallinity_%',
                                                           'Aver_crystallite_sizes_nm',
                                                           'Spherical_surface_Area_nm^2 '
                                                           '_TEM',
                                                           